In [ ]:
using Pkg
Pkg.activate(".")
using PlotlyJS
using DelimitedFiles
using Statistics
using Flux
using Optim
using LinearAlgebra: norm, dot, cross
using PyCall

skimage = pyimport("skimage")
trimesh = pyimport("trimesh")

σ(s) = 1/(1+exp(-s))

In [ ]:
R=2
r=1 
ax=0.1
ay=-0.1

f1(x, y, z) = (sqrt(x^2 + y^2) - R)^2 + (z - ax * x^2 - ay * y^2)^2 - r^2
f2(x,y,z) = (x-1)^2 + y^2 + z^2 - 1


In [ ]:
sd = 3.5
res = 150
box = [-sd,sd,-sd,sd,-sd,sd]
# create mesh grid for the box
x = range(box[1], box[2], length=res)
y = range(box[3], box[4], length=res)
z = range(box[5], box[6], length=res)

# create meshgrid
X = [i for i in x, j in y, k in z]
Y = [j for i in x, j in y, k in z]
Z = [k for i in x, j in y, k in z]

fs = f2.(X,Y,Z)

vs,fs,tmps = skimage.measure.marching_cubes(fs, 0, spacing=(1,1,1))
tmesh = trimesh.Trimesh(vertices=vs, faces=fs)
ps_even, = trimesh.sample.sample_surface_even(tmesh, 1000)

# plot vs

#sc = scatter3d(x=vs[:,1],y=vs[:,2],z=vs[:,3],mode="markers", marker_size=1)
sc_even = scatter3d(x=ps_even[:,1],y=ps_even[:,2],z=ps_even[:,3],mode="markers", marker_size=3)
mesh_graph = mesh3d(x=vs[:,1],y=vs[:,2],z=vs[:,3],i=fs[:,1],j=fs[:,2],k=fs[:,3],opacity=0.5,color="blue")
layout = Layout(width=600,height=600,scene_camera_eye=attr(x=1.25, y=1.25, z=1.25))
plot([mesh_graph, sc_even],layout)

In [ ]:
function compute_gradient(f)
    function ∇f(p)
        g = gradient(f, p[1], p[2], p[3])
        return [g[1], g[2], g[3]]
    end
    return ∇f
end

∇f1 =  compute_gradient(f1)

function compute_normal(f)
    function nf(p)
        g = gradient(f, p[1], p[2], p[3])
        return [g[1], g[2], g[3]] / norm([g[1], g[2], g[3]])
    end
    return nf
end

function get_plotly_lines(ps, ds)
    lines_x = []
    lines_y = []
    lines_z = []
    for i = 1:size(ps_even)[1]
        push!(lines_x, ps[i,1])
        push!(lines_x, ps[i,1] + ds[i,1])
        push!(lines_x, nothing)
        push!(lines_y, ps[i,2])
        push!(lines_y, ps[i,2] + ds[i,2])
        push!(lines_y, nothing)
        push!(lines_z, ps[i,3])
        push!(lines_z, ps[i,3] + ds[i,3])
        push!(lines_z, nothing)
    end
    return lines_x, lines_y, lines_z
end

a = ∇f1([1,2,3])
nf1 = compute_normal(f1)

normals = [nf1(ps_even[i,:]) for i = 1:size(ps_even)[1]]
normals = [[0,0,1] for i = 1:size(ps_even)[1]]


# conver normals to matrix
normals = hcat(normals...)'


lines_x, lines_y, lines_z = get_plotly_lines(ps_even, normals)

# plot lines 
lines = scatter3d(x=lines_x,y=lines_y,z=lines_z,mode="lines", marker_size=1)

plot([mesh_graph, lines],layout)



In [ ]:
size(ps_even)[1]